In [31]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import csv
from tqdm import tqdm

In [32]:
DATA_DIR = "/Volumes/SSD970/"

TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, "train_images")

In [33]:
dcm = pydicom.dcmread(os.path.join(TRAIN_IMAGES_DIR, f"1.2.826.0.1.3680043.20756/20.dcm"))
dcm

Dataset.file_meta -------------------------------
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.20756.1.20
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'PYDICOM 2.3.0'
-------------------------------------------------
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.20756.1.20
(0008, 0023) Content Date                        DA: '20220727'
(0008, 0033) Content Time                        TM: '182639.457634'
(0010, 0010) Patient's Name                      PN: '20756'
(0010, 0020) Patient ID                          LO: '20756'
(0018, 0050) Slice Thickness                     DS: '0.5'
(0020, 000d) Study Instance UID                  UI: 1.2.826.0.1

In [34]:
f"{dcm.PatientID}{dcm.InstanceNumber:04d}"

'207560020'

In [35]:
header = [
    'id',
    'PatientID',
    'StudyInstanceUID',
    'InstanceNumber',
    'Rows',
    'Columns',
    'SliceThickness',
    'ImagePositionPatientX',
    'ImagePositionPatientY',
    'ImagePositionPatientZ',
    'ImageOrientationPatientRowX',
    'ImageOrientationPatientRowY',
    'ImageOrientationPatientRowZ',
    'ImageOrientationPatientColX',
    'ImageOrientationPatientColY',
    'ImageOrientationPatientColZ',
    'BitsAllocated',
    'BitsStored',
    'HighBit',
    'PixelRepresentation',
    'WindowCenter',
    'WindowWidth',
    'RescaleIntercept',
    'RescaleSlope',
    'SamplesPerPixel',
    'PixelSpacingX',
    'PixelSpacingY',
]


In [36]:
def dcm_row(dcm):
    row = [
        f"{dcm.PatientID}{dcm.InstanceNumber:04d}",
        dcm.PatientID,
        dcm.StudyInstanceUID,
        dcm.InstanceNumber,
        dcm.Rows,
        dcm.Columns,
        dcm.SliceThickness,
        *dcm.ImagePositionPatient,
        *dcm.ImageOrientationPatient,
        dcm.BitsAllocated,
        dcm.BitsStored,
        dcm.HighBit,
        dcm.PixelRepresentation,
        dcm.WindowCenter,
        dcm.WindowWidth,
        dcm.RescaleIntercept,
        dcm.RescaleSlope,
        dcm.SamplesPerPixel,
        *dcm.PixelSpacing
    ]
    return row
dcm_row(dcm)

['207560020',
 '20756',
 '1.2.826.0.1.3680043.20756',
 '20',
 512,
 512,
 '0.5',
 '-116.0316',
 '-75.01602',
 '1980.4',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 16,
 16,
 15,
 1,
 '500.0',
 '2500.0',
 '0.0',
 '1.0',
 1,
 '0.358',
 '0.358']

In [45]:
dcm_file_paths = glob.glob(TRAIN_IMAGES_DIR + f"/**/*.dcm")

In [47]:
with open('meda_dcm_dataset.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for i, path in enumerate(dcm_file_paths):
        dcm = pydicom.dcmread(path)
        writer.writerow(dcm_row(dcm))

        if i == 5:
            break